# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

/root/miniconda3/envs/yeqing/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/miniconda3/envs/yeqing/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/root/miniconda3/envs/yeqing/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

乾卦是八卦之一，也是八宫图之一，它的卦象是由三个阳爻夹一个阴爻构成，象征着天、云和雷等自然现象。乾卦的卦辞为“元、亨、利、贞”，意味着“开始、发展、顺利、正道”。乾卦代表着阳、刚、强等特性，它象征着天地之间的力量和刚健不挠的精神。在五行中，乾卦属于木，代表着生长、勃发、创造等特性。在中医养生中，乾卦与呼吸、肝、魂等有关，与人体健康密切相关。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [8]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是八卦之一，它的卦象是由两个阴爻夹一个阳爻构成，象征着天下的万物，以及人类社会中的纷争和矛盾。讼卦的卦辞为“天、Meta、正、明”，意味着“向上、正确、明亮、和平”。

在这个卦象中，阳爻代表阳，阴爻代表阴。阳爻表示积极、刚强、刚正等特性，阴爻则表示消极、柔弱、温和等特性。因此，讼卦代表了阳与阴之间的矛盾和纷争。同时，这个卦也意味着只有通过正确的判断和明辨是非，才能达到和平、公正的状态。

在五行中，讼卦属于火，代表着光明、热情、勇气等特性。在中医养生中，讼卦与心脏、肺、皮肤等有关，与人体健康密切相关。


## 使用微调后的 ChatGLM3-6B(deepseek生成的数据集）

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [9]:
from peft import PeftModel, PeftConfig

epochs = 3
timestamp = "20250817_140600"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [10]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [11]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
在周易中，乾卦作为六十四卦之首，具有至高无上的地位。它由六个阳爻组成，象征纯阳至刚的宇宙本原力量。卦辞'元亨利贞'揭示了天道运行的四大特质：创始（元）、亨通（亨）、有利（利）、正固（贞）。《象传》'天行健，君子以自强不息'的经典阐释，将自然法则与人生修养完美统一。

乾卦以'天'为象，以'龙'为喻，展现了阳刚之气的运行轨迹。从初爻'潜龙勿用'到上爻'亢龙有悔'，完整呈现了事物发展的客观规律。其核心哲学在于：刚健而不失中正，进取而懂得节制。邵雍提醒'谨防太强'，傅佩荣指出'女子过刚宜慎重'，都体现了对阳刚过盛的警示。

在现实应用中，乾卦启示我们：
1. 事业上把握'飞龙在天'的机遇，同时警惕盛极而衰
2. 经商需遵循天道，戒骄戒躁
3. 求名应持续精进，克服自满
4. 婚恋讲究刚柔相济
5. 决策要秉持刚健中正之道

乾卦的现代价值在于：它既激励人们发扬自强不息的精神，又告诫要遵循客观规律，保持忧患意识，实现可持续发展。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250817_140600）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦作为六十四卦之首，具有至高无上的地位。它由六个阳爻组成，象征纯阳至刚的宇宙本原力量。卦辞'元亨利贞'揭示了天道运行的四大特质：创始（元）、亨通（亨）、有利（利）、正固（贞）。《象传》'天行健，君子以自强不息'的经典阐释，将自然法则与人生修养完美统一。

乾卦以'天'为象，以'龙'为喻，展现了阳刚之气的运行轨迹。从初爻'潜龙勿用'到上爻'亢龙有悔'，完整呈现了事物发展的客观规律。其核心哲学在于：刚健而不失中正，进取而懂得节制。邵雍提醒'谨防太强'，傅佩荣指出'女子过刚宜慎重'，都体现了对阳刚过盛的警示。

在现实应用中，乾卦启示我们：
1. 事业上把握'飞龙在天'的机遇，同时警惕盛极而衰
2. 经商需遵循天道，戒骄戒躁
3. 求名应持续精进，克服自满
4. 婚恋讲究刚柔相济
5. 决策要秉持刚健中正之道

乾卦的现代价值在于：它既激励人们发扬自强不息的精神，又告诫要遵循客观规律，保持忧患意识，实现可持续发展。


In [12]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示意味的卦象，由下卦坎（水）和上卦乾（天）组成。这一卦象象征着争讼和冲突，天水相违，流向相反，暗示着事理乖舛和矛盾对立。讼卦的核心哲学是：虽然初期可能有利可图，但最终会带来凶险，因此必须慎之又慎。

讼卦的卦辞指出：“有孚，窒惕，中吉，终凶。”这意味着即使有收获，也要保持警惕，因为事情可能在中期顺利，但最终会走向不利。占得此卦时，适合求助于有德之人，但不宜冒险行动（如涉大川）。

《象辞》进一步解释：“天与水违行，讼。”君子应从中领悟，在做事之初就要谨慎谋划，避免争讼。讼卦提醒人们，争讼非善事，务必戒惧，以和为贵。在事业、经商、求名等方面，此卦均强调退让、隐忍和公正的重要性，避免因逞强或贪图不义之利而招致灾祸。

邵雍解此卦为“天高水深，达远不亲”，寓意人际关系疏离，需谨慎退守。傅佩荣则从时运、财运、家宅等方面指出，此卦宜修身养性，防患于未然。总体而言，讼卦教导人们以柔克刚，知足常乐，避免陷入无谓的争端。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250817_140600）：
[gMASK]sop 周易中的讼卦是什么卦象， 在《周易》体系中，讼卦（坎上坤下）展现了一个充满警示意味的卦象。卦辞'利见大人'暗示着在困境中寻求突破的重要性。卦象中，兑为口（喜悦）、离为火（热情）和震为震的「元吉」等元素组合在一起，表明了积极行动的吉祥效果。

从卦辞到爻辞，都充满了对行动的强调。'利见大人'、'利西南，保其有'、'利见君'等语句，都强调了在困境中寻求突破的重要性。卦象也显示了'老老成成'的规律，暗示成熟稳重的人才能在困境中取得成功。

讼卦的核心哲学是：面对困境时，积极行动是唯一的吉祥之道。卦象中，兑宫（西南）和离宫（东南）都代表着积极的行动方向。同时，卦辞中的'大人'也暗示了需要有能力和智慧才能应对挑战。

在应用于现实生活方面，讼卦强调：当面临困境时，需要积极寻求突破，积极行动，而不是坐等困境消失。卦象显示了积极行动的吉祥效果，同时也提醒：积极行动并非盲目，需要在合适的时机采取合适的方法。


In [13]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
在《周易》体系中，师卦（䷆）具有独特而深刻的哲学意蕴。此卦由下坎（水）上坤（地）组成，形成'地中有水'的卦象，象征军事力量蕴藏于民众之中。卦辞'贞，丈人吉，无咎'强调军事行动必须坚守正道，由德高望重的统帅指挥方能吉祥。

从象数角度看，师卦作为坎宫归魂卦，揭示了'忧劳动众，变化无穷'的深层哲理。邵雍的解读指出，得此卦者将面临重重困难，需要'公正无私，排除万难'。傅佩荣则从实用角度提出时运、财运等方面的指导建议。

传统解卦着重分析其军事哲学：虽然'兵凶战危'，但圣人用兵是'不得已而为之'，关键在于'师出有名'和'顺乎形势'。卦象显示养兵聚众之态，暗示激烈的竞争环境。在事业、经商、求名等方面，都强调要刚柔并济、注重合作。

值得注意的是，《象传》'君子以容民畜众'的训诫，将军事智慧升华为治国之道，体现了周易'推天道以明人事'的思维特点。决策方面提示要发挥聪明才智，但需避免过度冒险，以中正之道为根本准则。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250817_140600）：
[gMASK]sop 师卦是什么？ 在《周易》体系中，师卦（䷆）具有独特而深刻的哲学意蕴。此卦由下坎（水）上坤（地）组成，形成'地中有水'的卦象，象征军事力量蕴藏于民众之中。卦辞'贞，丈人吉，无咎'强调军事行动必须坚守正道，由德高望重的统帅指挥方能吉祥。

从象数角度看，师卦作为坎宫归魂卦，揭示了'忧劳动众，变化无穷'的深层哲理。邵雍的解读指出，得此卦者将面临重重困难，需要'公正无私，排除万难'。傅佩荣则从实用角度提出时运、财运等方面的指导建议。

传统解卦着重分析其军事哲学：虽然'兵凶战危'，但圣人用兵是'不得已而为之'，关键在于'师出有名'和'顺乎形势'。卦象显示养兵聚众之态，暗示激烈的竞争环境。在事业、经商、求名等方面，都强调要刚柔并济、注重合作。

值得注意的是，《象传》'君子以容民畜众'的训诫，将军事智慧升华为治国之道，体现了周易'推天道以明人事'的思维特点。决策方面提示要发挥聪明才智，但需避免过度冒险，以中正之道为根本准则。


## 其他模型（错误数据或训练参数）

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [14]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, automade-dataset)"

In [15]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是《易经》六十四卦之首，由纯阳爻（六阳爻）组成，象征着天道的刚健与永恒运行。其卦辞'元、亨、利、贞'揭示了宇宙创始、亨通、有利、贞正四大特质，既是自然法则的体现，也是君子修身的最高准则。

《象传》'天行健，君子以自强不息'的经典阐释，将天体运行的不息刚健与人类应具备的精神品格相联系。北宋邵雍指出此卦'刚健旺盛'却需'谨防太强'的辩证智慧，傅佩荣则从时运角度强调'临事刚健'的实践意义。传统解卦特别警示'盛极必衰'的规律，要求持守中道。

在人生应用层面，乾卦展现多维指导价值：事业上象征'如日中天'却需戒骄戒躁；经商时提示把握机遇而保持商业道德；婚恋方面主张刚柔相济的平衡之道。其核心哲学在于：以天道为法，在自强不息中保持警醒，通过持续修德积累实现'飞龙在天'的理想境界。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 乾卦作为《易经》六十四卦之首，由纯阳爻（六阳爻）组成，象征着天道的刚健与永恒运行。其卦辞'元、亨、利、贞'揭示了宇宙创始、亨通、有利、贞正四大特质，既是自然法则的体现，也是君子修身的最高准则。

《象传》'天行健，君子以自强不息'的经典阐释，将天体运行的不息刚健与人类应具备的精神品格相联系。北宋邵雍指出此卦'刚健旺盛'却需'谨防太强'的辩证智慧，傅佩荣则从时运角度强调'临事刚健'的实践意义。传统解卦特别警示'盛极必衰'的规律，要求持守中道。

在人生应用层面，乾卦展现多维指导价值：事业上象征'如日中天'却需戒骄戒躁；经商时提示把握机遇而保持商业道德；婚恋方面主张刚柔相济的平衡之道。其核心哲学在于：以天道为法，在自强不息中保持警醒，通过持续修德积累实现'飞龙在天'的理想境界。


In [16]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

问题：地水师卦是什么？

原始输出：
{'name': "师卦（师卦：坤为师）是《周易》中卦象的一种，由下卦坤（代表地）和上卦震（代表雷）组成。卦辞'师出无名'揭示了 this 卦的特性：虽然有许多人想要领导或教导 others，但必须明确授权才能成功。", 'content': "\n《象传》深刻地描述了《师卦》的核心哲学：'师出无名'。这是对领导者的要求：必须有合法的来源，才能获得成功。就像军队需要纪律严明的指挥，必须听从合法的指挥者才能行动一样。\n\n《师卦》通过其卦象和卦辞，提供了对人类行为的深刻洞察：在领导或教导他人时，必须注意以下事项。\n\n1. 合法性：在领导或教导他人时，必须遵守法律法规，否则将无法获得成功。\n2. 适应性：领导者和教师需要适应不同的情况和环境，灵活应对。\n3. 谦卑：领导者和教师需要保持谦逊，不骄傲，以便能够持续学习和发展。\n4. 耐心：领导者和教师需要具备耐心，能够耐心地指导和等待。\n\n《师卦》总体来说，强调了领导者和教师在教导他人时的责任：通过合法性和适应性，激发他人的潜能，并在 patience 和谦逊 方面保持一致。"}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 地水师卦是什么？ 师卦的卦辞为：『师。贞，丈人吉，无咎』，意指在军事行动中，坚守正道并由德高望重的长者统帅，则能吉祥无灾祸。《象传》阐释其卦象为『地中有水』，象征君子应当效法大地包容江河的胸怀，容纳和养育民众。

从哲学层面解读，师卦（坤上坎下）展现了中国古代『寓兵于农』的治国智慧。坎为水象征险难，坤为地象征柔顺，二者结合揭示了军事力量的双重性：既是危险的利器，又可能是保境安民的保障。邵雍将其概括为『忧劳动众，变化无穷』，傅佩荣则从现实角度指出此卦提示人们需要包容和等待时机。

传统解卦强调：
1. 军事行动必须师出有名、顺应时势
2. 面临困境时需刚柔并济
3. 成功关键在于德才兼备的领导者
4. 凡事要避免投机取巧

现代启示：
- 事业：在激烈竞争中要善于合作，保持灵活
- 经商：大额投资需谨慎，遵守商业道德
- 处世：培养包容心，在困难中保持坚韧
- 决策：发挥才智但不冒进，以中正为原则

该卦整体呈现『以柔克刚』的辩证思维，强调在危机处理中既要果断行动又要保持道德操守，

In [17]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

问题：周易中的讼卦是什么

原始输出：
讼卦是《周易》中一个充满警示意味的卦象，由上乾（天）下坎（水）组成。卦辞'有孚，窒惕，中吉，终凶'揭示了争讼的本质：虽可能短期获利，但最终必招致凶险。天在上、水在下的卦象，象征着两种相背的力量，暗示着矛盾与冲突的不可避免性。

《象传》深刻指出'君子以做事谋始'的智慧，强调在事情开端就应谨慎谋划，从源头上避免争讼。邵雍'天高水深，达远不亲'的解读，进一步说明讼卦揭示的人际关系疏离状态。傅佩荣则从现实角度提出'功名受阻时不宜树敌'的处世建议。

传统解卦系统分析了讼卦的各个维度：
1. 事业方面：初期顺利但暗藏危机，强调'退让'的智慧
2. 经商之道：突出'和气生财'的东方商业伦理
3. 婚恋关系：提倡相互理解的温和相处之道
4. 人生决策：警示锋芒毕露的危险，主张知足常乐

讼卦与需卦构成综卦关系，共同阐释了人类面对需求与冲突时的不同应对策略，体现了《周易》'慎始慎终'的核心哲学思想。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 周易中的讼卦是什么卦象 讼卦是《周易》中一个充满警示意味的卦象，由上乾（天）下坎（水）组成。卦辞'有孚，窒惕，中吉，终凶'揭示了争讼的本质：虽可能短期获利，但最终必招致凶险。天在上、水在下的卦象，象征着两种相背的力量，暗示着矛盾与冲突的不可避免性。

《象传》深刻指出'君子以做事谋始'的智慧，强调在事情开端就应谨慎谋划，从源头上避免争讼。邵雍'天高水深，达远不亲'的解读，进一步说明讼卦揭示的人际关系疏离状态。傅佩荣则从现实角度提出'功名受阻时不宜树敌'的处世建议。

传统解卦系统分析了讼卦的各个维度：
1. 事业方面：初期顺利但暗藏危机，强调'退让'的智慧
2. 经商之道：突出'和气生财'的东方商业伦理
3. 婚恋关系：提倡相互理解的温和相处之道
4. 人生决策：警示锋芒毕露的危险，主张知足常乐

讼卦与需卦构成综卦关系，共同阐释了人类面对需求与冲突时的不同应对策略，体现了《周易》'慎始慎终'的核心哲学思想。
